In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import import_ipynb
from utils.dynamic_rnn import DynamicRNN

importing Jupyter notebook from /home/abc/Desktop/VisDial3/visdial/utils/dynamic_rnn.ipynb


In [ ]:
class LateFusionEncoder(nn.Module):
    def __init__(self, config, vocabulary):
        super().__init__()
        self.config = config

        # To embed the words in question & dialog history
        self.word_embed = nn.Embedding(
            len(vocabulary),
            config["word_embedding_size"],
            padding_idx=vocabulary.PAD_INDEX
        )

        # To apply LSTM on dialog history
        self.hist_rnn = nn.LSTM(
            config["word_embedding_size"],
            config["lstm_hidden_size"],
            config["lstm_num_layers"],
            batch_first=True,
            dropout=config["dropout"],
        )

        # To apply LSTM on question at time t
        self.ques_rnn = nn.LSTM(
            config["word_embedding_size"],
            config["lstm_hidden_size"],
            config["lstm_num_layers"],
            batch_first=True,
            dropout=config["dropout"],
        )

        # To avoid overfitting
        self.dropout = nn.Dropout(p=config["dropout"]) 

        # For reference resolution in question & dialog history
        self.hist_rnn = DynamicRNN(self.hist_rnn)
        self.ques_rnn = DynamicRNN(self.ques_rnn)

        # Project or convert 2048 image-feature vectors to 512 size lstm vectors
        self.image_features_projection = nn.Linear(
            config["img_feature_size"], config["lstm_hidden_size"]
        )

        # Calculating Attention weights
        self.attention_proj = nn.Linear(config["lstm_hidden_size"], 1)

        # Defining a fusion size by combining the image, question & history
        fusion_size = (
            config["img_feature_size"] + config["lstm_hidden_size"] * 2
        )

        # Combinning the img, ques & history
        self.fusion = nn.Linear(fusion_size, config["lstm_hidden_size"])

        # Initilizing Weights
        nn.init.kaiming_uniform_(self.image_features_projection.weight)
        nn.init.constant_(self.image_features_projection.bias, 0)
        nn.init.kaiming_uniform_(self.fusion.weight)
        nn.init.constant_(self.fusion.bias, 0)

